In [1]:
import json
import pandas as pd

In [2]:
with open("tagged.json") as user_file:
    file_contents = json.load(user_file)
tagged = pd.DataFrame(file_contents)

In [3]:
def preprocess(data, remove_judgement = True):
    # in: 2d table with prompts and resolution
    # out: tidied data
    # ends with 5 hashtags
    # remove capitalisation?
    # 
    if remove_judgement:
        data['completion'] = data['completion'].str.replace(r'[N,n,Y,y][T,t][A,a]', '', regex=True)
    data['completion'] = data['completion'].str.replace(r'#', '', regex=True)
    data['completion'] = data['completion'].str.replace(r'\s+', ' ', regex=True)
    return data

In [28]:
def correct_labels(data):
  data.loc[(data["completion"].str.lower()).str.startswith(' definitely nta'), 'label'] = 0
  return data

In [32]:
nta = pd.read_json("NTA_Dataset.jsonl", lines=True)
yta = pd.read_json("YTA_Dataset.jsonl", lines=True)
nta['label'] = 0
yta['label'] = 1
yta = correct_labels(yta)

In [33]:
data = pd.concat([nta,yta])
data = preprocess(data)

In [34]:
full_tagged = pd.DataFrame()
data["reason"] = -5
for index, row in tagged.iterrows():
    tagged_completion =  row["completion"]
    data.loc[data["completion"]==tagged_completion, "reason"] = row["reason"]
    #this_row["reason"] = row["reason"]
    #full_tagged = pd.concat([full_tagged,this_row])
full_tagged = data.loc[data["reason"]!=-5]

In [35]:
full_tagged

,prompt,completion,label,reason
66,So my Mum and Dad moved about 5 hours away fro...,Your parents have every right to choose a gua...,0,1
103,Six months ago my now boyfriend broke up with ...,Your bf is actively lying to and emotionally ...,0,6
134,"This story starts around February 2020,\nSo cl...",You shared the screenshot and had a private c...,0,6
141,My school has decided to continue with a gradu...,You logic is reasonable. If your bio-dad does...,0,1
191,My “best” friend of about 8 years is pregnant ...,"Yes, mood swings while pregnant are absolutel...",0,11
...,...,...,...,...
2284,My husband (28) and I (43) had a small birthda...,big time >My husband was humiliated and somet...,1,-1
2310,Yesterday it was my daughter's 15th birthday a...,and the piercer you went to was irresponsible...,1,0
2347,"I work for this factory, i'm pretty much a for...",This is your sons income and you should be ha...,1,8
2359,My parents invited My brother (22M) for dinner...,Coming out is such a raw and vulnerable mome ...,1,3


In [36]:
full_tagged.to_json("tagged_data.json")